In [2]:
from beam_solver.beam import Beam

class Cantilever_Beam(Beam):    
    """
    Creates a customizable beam object, also includes methods to present the Freebody Diagram
    and perform calculations as desired
    """
    
    x_pos = 2.0
    y_pos = 3.0
    p0 = Point(x_pos, y_pos)
    scaled_length= 10
    
    def __init__(self, length, height):
        """
        Initializing the beam properties, additional properties will be added on top
        as the user desires.
        length: [m]
        height: [m]
        """
        
        self.length = length
        self.height = 1
        self.beam_height = height
        self.main = Rectangle(self.p0, self.scaled_length, height).set_fill_pattern(Style.FillPattern.UP_LEFT_TO_RIGHT)
        self.composition={"main": self.main}
        #self.force_dictionary={}
        self.force_dictionary={'magnitude':[],'position':[],'type':[]}
        #self.moment_dictionary={}
        self.moment_dictionary={'magnitude':[],'position':[],'name':[]}
        #self.support_dictionary={}
        self.support_dictionary={'magnitude':[],'position':[],'type':[]}
        
        self.UDL_dictionary = {'magnitude':[],'position':[],'distributed_load':[], 'end_position':[]}

        super().__init__(length, height)

        clamped = Rectangle(self.p0 - Point(1, 0) - Point(0, 2 * 1), 1, 5 * 1).set_fill_pattern(
        Style.FillPattern.UP_LEFT_TO_RIGHT)
        self.composition['clamped']= clamped
        reaction_force= Force("$R_1$",self.p0 - Point(0, 2 * self.height),self.p0)
        reaction_force.set_line_width(3).set_line_color(Style.Color.BLACK)
        self.composition['Reaction']= reaction_force
        reaction_moment = Moment("$RM_1$",center=self.p0 + Point(0, self.height/ 2),
                                 radius=self.height/ 2)
        reaction_moment.line_color = "black"
        self.composition['Reaction_Moment']= reaction_moment
    def calculate_forces(self):
        """
            Calculates the support reactions for the beam.
            Solves the system of equations involved from creating a sum of forces,
            and sum of moments.
        """

        force= pd.DataFrame.from_dict(self.force_dictionary)
        moment= pd.DataFrame.from_dict(self.moment_dictionary)
        support = pd.DataFrame.from_dict(self.support_dictionary)
        #Rearranging support dataframe so 'Fixed is always first'
        support= (support.sort_values('type')).reset_index(drop=True)
        fixed_support_position = 0
        
        #Fixed Support Position is the beginning of the lever arm. The lever arm
        #is the distance from some force to the fixed support
        #
        sum_force_reactions = -1*force['magnitude'].sum()
        
        #Calculating the moment generated by all forces placed along the beam.
        force['moment'] = (force['position'] - fixed_support_position)*force['magnitude']
        sum_moments_and_couples = -1*force['moment'].sum() - moment['magnitude'].sum()
        
        self.moment_dictionary['magnitude'].append(sum_moments_and_couples)
        self.moment_dictionary['position'].append(0)
        self.moment_dictionary['name'].append("M")
    
        #Solving for support reactions and giving values to the support dictionary.
        self.support_dictionary['magnitude'].append(sum_force_reactions)
        self.support_dictionary['position'].append(0)
        self.support_dictionary['type'].append('fixed')


ModuleNotFoundError: No module named 'beam_solver'